In [1]:
import numpy as np
from jesse import helpers, research

_, trading_1m = research.get_candles(
    "Binance Perpetual Futures",
    "BTC-USDT",
    "1m",
    helpers.date_to_timestamp("2020-01-01"),
    helpers.date_to_timestamp("2025-04-20"),
    warmup_candles_num=0,
    caching=False,
    is_for_jesse=False,
)

trading_1m.shape


(2787840, 6)

In [2]:
np.save("data/btc_1m.npy", trading_1m)

In [3]:
import numpy as np

candles_1m = np.load("data/btc_1m.npy")
candles_1m.shape

(2787840, 6)

In [4]:
from jesse import utils
from tqdm.auto import tqdm

tqdm.pandas()

df = utils.numpy_candles_to_dataframe(candles_1m)
for i in range(1, 10):
    df[f"log_ret_{i}"] = np.log(df["close"]) - np.log(df["close"].shift(i))
print(df.shape)
df.tail()

(2787840, 15)


,date,open,close,high,low,volume,log_ret_1,log_ret_2,log_ret_3,log_ret_4,log_ret_5,log_ret_6,log_ret_7,log_ret_8,log_ret_9
2025-04-19 23:55:00,2025-04-19 23:55:00,85085.2,85093.1,85093.1,85085.1,21.587,0.000093,-0.000222,-0.000214,-0.000166,-0.000165,-0.000166,-0.000122,-0.000008,0.000183
2025-04-19 23:56:00,2025-04-19 23:56:00,85093.0,85085.3,85093.1,85085.2,10.507,-0.000092,0.000001,-0.000314,-0.000306,-0.000257,-0.000256,-0.000257,-0.000214,-0.000100
2025-04-19 23:57:00,2025-04-19 23:57:00,85085.2,85056.5,85085.3,85056.4,21.570,-0.000339,-0.000430,-0.000337,-0.000652,-0.000644,-0.000596,-0.000595,-0.000596,-0.000552
2025-04-19 23:58:00,2025-04-19 23:58:00,85056.5,85024.7,85056.5,85023.0,60.928,-0.000374,-0.000712,-0.000804,-0.000711,-0.001026,-0.001018,-0.000970,-0.000969,-0.000970
2025-04-19 23:59:00,2025-04-19 23:59:00,85024.7,85046.9,85047.0,85022.0,62.258,0.000261,-0.000113,-0.000451,-0.000543,-0.000450,-0.000765,-0.000757,-0.000709,-0.000708


In [5]:
from pandarallel import pandarallel
from tqdm.auto import tqdm

from custom_indicators.toolbox.entropy.apen_sampen import sample_entropy

pandarallel.initialize(progress_bar=False)

for i in tqdm(range(1, 10)):
    for j in [10, 20, 30, 40, 50, 100, 200]:
        df[f"{i}_sampEn_{j}"] = (
            df[f"log_ret_{i}"]
            .rolling(window=j)
            .parallel_apply(lambda x: sample_entropy(x.to_numpy()))
        )
        df[f"{i}_sampEn_{j}"].fillna(df[f"{i}_sampEn_{j}"].max(), inplace=True)

df.tail()


INFO: Pandarallel will run on 14 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


  0%|          | 0/9 [00:00<?, ?it/s]

,date,open,close,high,low,volume,log_ret_1,log_ret_2,log_ret_3,log_ret_4,...,8_sampEn_50,8_sampEn_100,8_sampEn_200,9_sampEn_10,9_sampEn_20,9_sampEn_30,9_sampEn_40,9_sampEn_50,9_sampEn_100,9_sampEn_200
2025-04-19 23:55:00,2025-04-19 23:55:00,85085.2,85093.1,85093.1,85085.1,21.587,0.000093,-0.000222,-0.000214,-0.000166,...,0.396536,0.166623,0.152082,1.098612,0.609064,0.346716,0.339507,0.356977,0.174000,0.160267
2025-04-19 23:56:00,2025-04-19 23:56:00,85093.0,85085.3,85093.1,85085.2,10.507,-0.000092,0.000001,-0.000314,-0.000306,...,0.414495,0.171533,0.151644,inf,0.632523,0.344504,0.360885,0.374115,0.176735,0.158619
2025-04-19 23:57:00,2025-04-19 23:57:00,85085.2,85056.5,85085.3,85056.4,21.570,-0.000339,-0.000430,-0.000337,-0.000652,...,0.377763,0.171830,0.151597,1.704748,0.871839,0.325020,0.354310,0.359719,0.176658,0.158056
2025-04-19 23:58:00,2025-04-19 23:58:00,85056.5,85024.7,85056.5,85023.0,60.928,-0.000374,-0.000712,-0.000804,-0.000711,...,0.373682,0.171552,0.151947,0.628609,0.553385,0.345159,0.365210,0.352411,0.179886,0.158180
2025-04-19 23:59:00,2025-04-19 23:59:00,85024.7,85046.9,85047.0,85022.0,62.258,0.000261,-0.000113,-0.000451,-0.000543,...,0.378797,0.173602,0.152287,0.510826,0.449917,0.347401,0.378797,0.363859,0.182527,0.158754


In [6]:
df.to_parquet("data/btc_1m_entropy.parquet")


In [2]:
import pandas as pd

df = pd.read_parquet("data/btc_1m_entropy.parquet")

In [17]:
# import plotly.express as px
# from plotly.subplots import make_subplots

# LENGTH = 40

# rows = [f"{i}_sampEn_{LENGTH}" for i in range(1, 10)]
# fig = make_subplots(rows=len(rows), cols=1, subplot_titles=rows, shared_xaxes=True)

# for i, name in enumerate(rows):
#     hist = px.histogram(df[name].tolist(), nbins=1000)
#     fig.add_trace(hist.data[0], row=i + 1, col=1)

# fig.update_layout(height=len(rows) * 200, showlegend=False)
# fig.show()

In [2]:
import numpy as np
import pandas as pd
from lightweight_charts import Chart

target = "5_sampEn_10"

df = pd.read_parquet("data/btc_1m_entropy.parquet").replace([np.inf, -np.inf], np.nan)
df[target] = df[target].fillna(df[target].max())
# df[target] = df["sampEn_20"] - df["sampEn_20"].shift(1)
df_draw = df.iloc[-10000:]

chart = Chart(inner_width=1, inner_height=0.8)
chart2 = chart.create_subchart(position="bottom", width=1, height=0.2, sync=True)

chart.set(df_draw[["date", "open", "high", "low", "close", "volume"]])

line = chart2.create_line(name=target)
line.set(df_draw[[target]])

chart.show(block=True)


RuntimeError: asyncio.run() cannot be called from a running event loop